In [1]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam

In [2]:
def create_input():
    X = []
    y = []
    df = pd.read_csv("classification_data/explore_img/explore_data.csv")
    shuffled_df = df.sample(len(df))

    print(shuffled_df.head)
    s = 0
    for entry in shuffled_df.values:
        s += 1
        # print(s)
        entry_img = cv2.imread(f"classification_data/explore_img/{entry[0]}")

        prepared_y = np.delete(entry,0) * 3
        print(f'before: {prepared_y}')
        if prepared_y[0] <= 960:
            prepared_y[0] =  -(960 - prepared_y[0])
        else:
            prepared_y[0] = prepared_y[0]-960
            
        if prepared_y[1] <= 540:
            prepared_y[1] = - (540 - prepared_y[1])
        else:
            prepared_y[1] = prepared_y[1] - 540
        print(prepared_y)
        
        X.append(entry_img)
        y.append(prepared_y)
        
    
    X = np.asarray(X)
    y = np.asarray(y).astype(np.float32)
    return X, y


In [3]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer=Adam())

    return model

model = compile_model()

In [4]:
model.compile(optimizer='adam',
              loss='mean_squared_error')

In [10]:
X, y = create_input()

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, y_train.shape)


<bound method NDFrame.head of             filename    x    y
194  explore_194.jpg  177  242
211  explore_211.jpg  254  310
290  explore_290.jpg   67  190
195  explore_195.jpg  122   68
187  explore_187.jpg  521  108
..               ...  ...  ...
381  explore_381.jpg  126  142
120  explore_120.jpg  546  105
42    explore_42.jpg  429   60
274  explore_274.jpg  186   32
149  explore_149.jpg  111   75

[403 rows x 3 columns]>
before: [531 726]
[-429 186]
before: [762 930]
[-198 390]
before: [201 570]
[-759 30]
before: [366 204]
[-594 -336]
before: [1563 324]
[603 -216]
before: [1611 219]
[651 -321]
before: [1131 936]
[171 396]
before: [1629 756]
[669 216]
before: [639 252]
[-321 -288]
before: [516 837]
[-444 297]
before: [609 810]
[-351 270]
before: [645 771]
[-315 231]
before: [606 705]
[-354 165]
before: [1701 504]
[741 -36]
before: [180 555]
[-780 15]
before: [360 639]
[-600 99]
before: [1296 168]
[336 -372]
before: [1185 123]
[225 -417]
before: [1362 126]
[402 -414]
before: [480 786]


In [6]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_val, y_val))


Epoch 1/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - loss: 3389673.7500 - val_loss: 312863.4688
Epoch 2/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 843ms/step - loss: 243402.7500 - val_loss: 198242.4531
Epoch 3/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 852ms/step - loss: 172751.6719 - val_loss: 151730.2344
Epoch 4/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 882ms/step - loss: 160992.0938 - val_loss: 145286.2656
Epoch 5/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 829ms/step - loss: 156161.5781 - val_loss: 140004.7188
Epoch 6/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 7s 812ms/step - loss: 144794.4219 - val_loss: 147473.0781
Epoch 7/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 774ms/step - loss: 129132.8828 - val_loss: 189636.1406
Epoch 8/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 782ms/step - loss: 150154.9375 - val_loss: 135782.0000
Epoch 9/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 795ms/step - loss: 118053.9375 - val_loss: 144915.0938
Epoch 10/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 796ms/step - loss: 109525.7500 - val_loss: 127411.0781
Epoch 11/50
8/8 ━━━━━━━━━━━━━━━━━━━━ 6s 772ms/step - loss: 8

In [7]:
model.save('mouse_explore_fhd_normalized.h5')

In [14]:
model_rap = tf.keras.models.load_model('mouse_explore_fhd_normalized2.h5')


for n in range(20):
    n = n +20
    x_real, y_real = int(y_val[n][0]), int(y_val[n][1])
    
    if x_real <= 960:
        x_real =  960 + x_real
    else:
        x_real = x_real + 960
        
    if y_real <= 540:
        y_real = 540 + y_real 
    else:
        y_real = y_real + 540
    
    img = np.asarray(X_val[n])

    tbp = np.expand_dims(img, axis=0)
    img = cv2.resize(img,(1920,1080))
    predictions = model_rap.predict(tbp)
    x, y = int(predictions[0][0]), int(predictions[0][1])
    
    if x <= 960:
        x =  960 + x
    else:
        x = x + 960
        
    if y <= 540:
        y = 540 + y 
    else:
        y = y + 540
    
    print(x, y, "\n",x_real, y_real)
    
    cv2.circle(img, (x,y), 10, (0,0,255), -1)
    cv2.circle(img, (x_real, y_real), 20, (255,255,255), -1)
    
    cv2.imshow('ss', img)
    cv2.waitKey(0) & 0xFF
cv2.destroyWindow('ss')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
202 196 
 108 153
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1021 255 
 378 426
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
822 343 
 939 69
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1328 165 
 1425 132
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1732 669 
 1836 702
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
811 463 
 429 165
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
814 305 
 783 378
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
885 509 
 1137 219
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
889 596 
 1446 261
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
806 528 
 360 639
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1142 239 
 1017 51
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1044 210 
 873 150
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
687 339 
 531 726
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1179 852 
 1020 945
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1407 207 
 1263 87
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1179 852 
 963 987
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1393 137 
 1332 105
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/

In [11]:
model_rap = tf.keras.models.load_model('mouse_explore_fhd_normalized.h5')
model_rap.compile(optimizer='adam',
              loss='mean_squared_error')


model_rap.fit(X_train, y_train, epochs=70, validation_data=(X_val, y_val))
model_rap.save('mouse_explore_fhd_normalized2.h5')

Epoch 1/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 735715.8750 - val_loss: 131288.5156
Epoch 2/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 14s 1s/step - loss: 125163.2031 - val_loss: 122923.0703
Epoch 3/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 110525.4766 - val_loss: 100067.1719
Epoch 4/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 83076.1016 - val_loss: 80139.3359
Epoch 5/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 66418.4766 - val_loss: 83101.9922
Epoch 6/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 50298.2266 - val_loss: 69796.7031
Epoch 7/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 32745.4199 - val_loss: 67354.6328
Epoch 8/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 21479.2988 - val_loss: 81441.3828
Epoch 9/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 12s 1s/step - loss: 30637.0371 - val_loss: 72336.4219
Epoch 10/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 11s 1s/step - loss: 22397.6641 - val_loss: 73763.8359
Epoch 11/70
11/11 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - loss: 16824.0508 - v